Netflix es una empresa de entretenimiento y una plataforma de streaming estadounidense. El equipo de tecnología de la compañía está interesado en estudiar las dinámicas de ocupación de la plataforma, la cual permite a los usuarios ver series y películas sin publicidades en un dispositivo con conexión a internet. Netflix se encuentra en funcionamiento durante las 24 horas del día, todos los días de la semana y cuenta con una capacidad de $C$ servidores. Cuando un usuario se conecta a la plataforma ocupa un servidor.

Con base en estudios de desempeño, realizados por el equipo de tecnología, se ha podido determinar que el número de usuarios que se conectan a la plataforma sigue un Proceso de Poisson con tasa $\lambda$ usuarios por hora. La plataforma evita el ingreso de los usuarios que intentan conectarse después de haber alcanzado la capacidad del sistema (en caso de que $C$ servidores se encuentren ocupados).

Pese a la gran acogida de Netflix por parte de los usuarios, el equipo de tecnología ha recibido numerosas quejas a causa de la ocupación de la plataforma. Los usuarios expresan que al intentar conectarse a la plataforma los servidores casi siempre se encuentran ocupados, y además, sospechan que muchos de ellos en realidad no están siendo usados sino que corresponden a usuarios que se conectaron y se quedaron dormidos. Con el fin de abordar este problema, el equipo de tecnología ha desarrollado un programa que revisa la ocupación de la plataforma al final de cada hora y escoge **un usuario** al azar (de los que se encuentran conectados) para solicitarle una confirmación de que se encuentra utilizando la plataforma. Con una probabilidad de $p$ el usuario confirma que se encuentra usando Netflix, por lo que el programa le permite seguir conectado; mientras que con la probabilidad restante el usuario no envía ninguna confirmación y es desconectado de la plataforma.

En caso de que todos los servidores ($C$) se encuentren ocupados al final de la hora $n$, la plataforma colapsa con una probabilidad de $r$ y desconecta a todos los usuarios que estaban conectados. Con la probabilidad restante, la plataforma no colapsa y simplemente continúa con el sistema de escoger un usuario al azar para preguntarle si está o no conectado al final de la hora siguiente. Además, durante la siguiente hora el programa impide que más usuarios se conecten, con el fin de descongestionar la plataforma.

**Nota:** el colapso de la plataforma solamente puede suceder (con probabilidad $r$) cuando el programa revisa el número de usuarios conectados al final de la hora $n$ y este es igual a $C$.

$$
X_n: \text{Número de usuarios conectados a Netflix al final de la hora } n, \text{ justo antes de que el programa realice la revisión de la ocupación}
$$

$$
S_X = \{ 0, 1, 2, \dots, C \}
$$

Se definen las siguientes variables aleatorias:

$$
Y: \text{número de usuarios que se conectan a Netflix en una hora}
$$

$$
Y \sim \text{Poisson}(\lambda, t=1)
$$

$$
P(Y = y) = \frac{e^{-\lambda} \cdot \lambda^y}{y!}
$$

$$
Z = \begin{cases}
1 & \text{si el usuario seleccionado al azar confirma que se encuentra conectado.}\\[1mm]
0 & \text{si el usuario seleccionado al azar no envía la confirmación.}
\end{cases}
$$

$$
Z \sim \text{Bernoulli}(p)
$$

Probabilidades de transición:

$$
\mathbb{P}_{i \rightarrow j} =
\begin{cases}
P(Y=j) & i=0,\quad 0 \leq j < C,\\[2mm]
P(Y \geq C) & i=0,\quad j=C,\\[2mm]
(1-p) \cdot P(Y=0) & j=i-1,\quad 0 < i < C,\\[2mm]
p \cdot P(Y=j-i) + (1-p) \cdot P(Y=j-i+1) & i \leq j,\quad 0 < i < C,\quad 0 < j < C,\\[2mm]
p \cdot P(Y \geq j-i) + (1-p) \cdot P(Y \geq j-i+1) & j=C,\quad 0 < i < C,\\[2mm]
r & i=C,\quad j=0,\\[2mm]
(1-r) \cdot (1-p) & i=C,\quad j=C-1,\\[2mm]
(1-r) \cdot p & i=C,\quad j=C,\\[2mm]
0 & \text{d.l.c.}
\end{cases}
$$

### Preguntas:

Asuma que $C=100,\ \lambda=8,\ p=0.8,\ r=0.1$

Construya la matriz de transición $P$ de la cadena de Markov, luego cree la cadena de Markov utilizando la librería `jmarkov`.



In [2]:
# Imports
import numpy as np
import math 
from jmarkov.dtmc import dtmc
from scipy.stats import poisson


In [3]:
# Parameters
C = 100 # Numero de servidores
lmbda = 8 # Tasa de llegada de clientes
p = 0.8 # Probabilidad de que un cliente confirme que sigue en Netflix
r = 0.1 # Probabilidad de que la plataforma colapse si todos los servidores estan ocupados al final de la hora n

In [6]:
estados = np.array([i for i in range(0,C+1)])  # Estados del sistema: 0 a C servidores ocupados   
matriz_transicion = np.zeros((len(estados),len(estados)))  # Matriz de transicion inicializada en ceros


# poisson PDF = poisson.pmf(k, mu)
# poisson CDF = poisson.cdf(k, mu)

for i in range(len(estados)):
    for j in range(len(estados)):
        if i == 0 and (0<=j<C):
           matriz_transicion[i,j] = poisson.pmf(j, lmbda)
        elif i == 0 and j == C:
           matriz_transicion[i,j] = 1-poisson.cdf(C-1,lmbda)
        elif j == i-1 and 0<i<C:
            matriz_transicion[i,j]=(1-p)*poisson.pmf(0,lmbda)
        elif i <= j and 0 < i < C and 0 < j < C:
            matriz_transicion[i,j]=p*poisson.pmf(j-i,lmbda) +(1-p)*poisson.pmf(j-i+1,lmbda)
        elif j == C and 0 < i < C:
            matriz_transicion[i,j] = p*(1-poisson.cdf(j-i-1,lmbda))+(1-p)*(1-poisson.cdf(j-i-1+1,lmbda))
        elif i == C and j == 0:
            matriz_transicion[i,j] = r
        elif i == C and j == C-1:
            matriz_transicion[i,j] = (1-r)*(1-p)
        elif i == C and j == C:
            matriz_transicion[i,j]=(1-r)*p
        else:
            matriz_transicion[i,j]=0

np.round(matriz_transicion, 4)
            

array([[3.000e-04, 2.700e-03, 1.070e-02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e-04, 8.000e-04, 4.300e-03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 1.000e-04, 8.000e-04, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 8.000e-04, 4.300e-03,
        9.948e-01],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e-04, 8.000e-04,
        9.991e-01],
       [1.000e-01, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.800e-01,
        7.200e-01]])

In [7]:
cadena = dtmc(matriz_transicion, estados)